In [ ]:
import os
import sys
import pandas as pd
from bs4 import BeautifulSoup
from pathlib import Path
import xlsxwriter

# Get Header Data

In [ ]:
def getHeader(file):
    with open(file, encoding='utf8') as infile:
        soup = BeautifulSoup(infile,'html.parser')
        headerList = []

        # Get Header Data
        header = soup.find_all("table")[2].find("tr")
        for items in header:
            try:
                headerList.append(items.get_text())
            except:
                continue
        # print("List Header : :",headerList)
        return headerList

# Get Main Data

In [ ]:
def getMainData(file, header):
    with open(file, encoding='utf8') as infile:
        soup = BeautifulSoup(infile,'html.parser')
        data = []

        # Get Main Data
        HTML_data = soup.find_all("table")[2].find_all("tr")[1:]
        for element in HTML_data:
            sub_data = []
            for sub_element in element:
                try:
                    sub_data.append(sub_element.get_text())
                except:
                    continue
            data.append(sub_data)

        dataframe = pd.DataFrame(data=data, columns=header)

        return dataframe

# Get Indicator

In [ ]:
# Get sum of indicator data
def findVariable(header, data, indicator):        
    result_indicator = 0
    for column in header:
        for variable in indicator:
            if column == variable:
                # print(column, "Detect")
                dataframe = data[column].value_counts().sum()
                blank = sum(data[column] == '')
                result_indicator = dataframe-blank
                # print('Jumlah Data :', sum_data,"\n")
    return result_indicator
# institute_name_sum = findVariable(institute_name)

In [ ]:
# Directory Information
institute_name = ["kampus", "campus", "universitas", "institut", "sekolah", "madrasah", "university", "school",]
accreditation = ['akreditasi', 'accreditation']
institute_address = ['alamat', 'address']
institute_number = ['number', 'no', 'phone']

# Educational Information
educational_history = ['riwayat_pendidikan', 'educational_history']
educational_medicine = ['catatan_rekam_medis', 'medicine']
educational_grade =[ 'grade', 'nilai']

# Personally Identifiable
personal_number = ['number', 'no', 'phone']
personal_address = ['address', 'alamat']
personal_birthday = ['birthday', 'bornday']
personal_religion = ['agama', 'religion']
personal_gender = ['gender', 'jenis_kelamin']
personal_parent = ['orang_tua', 'parent', 'parent_name', 'parents']
personal_photo = ['photo', 'foto']

In [ ]:
# Get sum of a coumn data on every file
total_institute_name = total_accreditation = total_institute_address = total_institute_number = 0
total_educational_history = total_educational_medicine = total_educational_grade = 0
total_personal_number = total_personal_address = total_personal_birthday = total_personal_religion = total_personal_gender = total_personal_parent = total_personal_photo = 0

for file in Path("website").iterdir():
    # Get Main Data
    header = getHeader(file)
    data = getMainData(file, header)
    
    # # Get sum of indicator
    # sumVarible = findVariable(header, data, institute_name)

    if file.is_file():
        # Directory Information
        total_institute_name += findVariable(header, data, institute_name)
        total_accreditation += findVariable(header, data, accreditation)
        total_institute_address += findVariable(header, data, institute_address)
        total_institute_number += findVariable(header, data, institute_number)

        # Educational Information
        total_educational_history += findVariable(header, data, educational_history)
        total_educational_medicine += findVariable(header, data, educational_medicine)
        total_educational_grade += findVariable(header, data, educational_grade)

        # Personally Identifiable
        total_personal_number += findVariable(header, data, personal_number)
        total_personal_address += findVariable(header, data, personal_number)
        total_personal_birthday += findVariable(header, data, personal_birthday)
        total_personal_religion += findVariable(header, data, personal_religion)
        total_personal_gender += findVariable(header, data, personal_gender)
        total_personal_parent += findVariable(header, data, personal_parent)
        total_personal_photo += findVariable(header, data, personal_photo)

print("Directory Information")
print("Nama Institusi : ", total_institute_name)
print("Akreditasi : ", total_accreditation)
print("Alamat Institusi : ", total_institute_address)
print("Nomor Telepon : ", total_institute_number,"\n")

print("Educational Information")
print("Riwayat Pendidikan : ", total_educational_history)
print("Catatan Rekam Medis : ", total_educational_medicine)
print("Nilai : ", total_educational_grade,"\n")

print("Personally Identifiable")
print("No Hp Siswa : ", total_personal_number)
print("Alamat Rumah Siswa : ", total_personal_address)
print("Tempat Tanggal Lahir : ", total_personal_birthday)
print("Agama : ", total_personal_religion)
print("Jenis Kelamin : ", total_personal_gender)
print("Nama Orang Tua : ", total_personal_parent)
print("Foto Diri : ", total_personal_photo)

# End Result

In [ ]:
# End Result
directory_information = total_institute_name + total_accreditation + total_institute_address + total_institute_number
educational_information = total_educational_history + total_educational_medicine + total_educational_grade
personally_identifiable = total_personal_number + total_personal_address + total_personal_birthday + total_personal_religion + total_personal_gender + total_personal_parent + total_personal_photo

print("Directory Information : ", directory_information)
print("Educational Information : ", educational_information)
print("Personally Identifiable : ", personally_identifiable)

# Preparation Data for Ecel

In [ ]:
# Preparation Data
directory_indicator = ["Nama Institusi","Akreditasi","Alamat Institusi","Nomor Telepon"]
educational_indicator = ["Riwayat Pendidikan","Catatan Rekam Medis","Nilai"]
personally_indicator = ["No HP Siswa","Alamat Rumah Siswa","Tempat Tanggal Lahir","Agama","Jenis Kelamin","Nama Orang Tua","Foto Diri"]
result_directory_indicator = [total_institute_name,total_accreditation,total_institute_address,total_institute_number]
result_educational_indicator = [total_educational_history,total_educational_medicine,total_educational_grade]
result_personally_indicator = [total_personal_number,total_personal_address,total_personal_birthday,total_personal_religion,total_personal_gender,total_personal_parent,total_personal_photo]

In [ ]:
def merge(list1,list2):
    merged_list = tuple(zip(list1,list2))
    return merged_list

In [ ]:
directory = merge(directory_indicator,result_directory_indicator)
educational = merge(educational_indicator,result_educational_indicator)
personally = merge(personally_indicator,result_personally_indicator)

# Excel Output

In [ ]:
# Create a workbook and add a worksheet.
workbook = xlsxwriter.Workbook('Website.xlsx')
worksheet = workbook.add_worksheet()

# Format Cell
align_format = workbook.add_format({'align': 'center'})
border_format = workbook.add_format({'border':1})
worksheet.conditional_format( 'A1:F10' , {'type' : 'blanks','format' : border_format} )
worksheet.conditional_format( 'A1:F10' , {'type' : 'no_blanks','format' : border_format} )

In [ ]:
def excelOutput(category,col_indicator,col_data,total):
    row = 1
    row_total = 9
    
    # Sub Header
    worksheet.write(row, col_indicator, 'Indikator')
    worksheet.write(row, col_data, 'Data', align_format)
    row += 1

    for indicator, data in category:
        worksheet.write(row, col_indicator, indicator)
        worksheet.write(row, col_data, data, align_format)
        row += 1 

    worksheet.write(row_total, col_data, total, align_format)

In [ ]:
# Column Size
worksheet.set_column('A:A', 15)
worksheet.set_column('C:C', 20)
worksheet.set_column('E:E', 22)

# Header
worksheet.merge_range('A1:B1', 'Directory Information', align_format)
worksheet.merge_range('C1:D1', 'Educational Information', align_format)
worksheet.merge_range('E1:F1', 'Personally Identifiable', align_format)

# Main Data
excelOutput(directory,0,1,'=SUM(B1:B7)')
excelOutput(educational,2,3,'=SUM(D1:D7)')
excelOutput(personally,4,5,'=SUM(F1:F7)')

In [ ]:
# Total Data & Save File
worksheet.write(9, 0, 'Total', align_format)
workbook.close()